In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 30bc0446-b872-4379-bd1f-b6453eb207f9, 3, Finished, Available)

In [2]:
%%sql
create table if not exists TMDW_FinalProj.gld_Cust
(
  Cust_ID	string	,
Cust_Name	string	,
Segment	string	,
City	string	,
State	string	,
Country	string	,
Region	string	,
Created_TS	timestamp	,
Modified_TS	timestamp	  

)
using delta

StatementMeta(, 30bc0446-b872-4379-bd1f-b6453eb207f9, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Maximum_Date=spark.sql("""select coalesce(max(Modified_TS),"1900-01-01") from TMDW_FinalProj.gld_Cust""").first()[0]

StatementMeta(, 30bc0446-b872-4379-bd1f-b6453eb207f9, 5, Finished, Available)

In [4]:
Maximum_Date

StatementMeta(, 30bc0446-b872-4379-bd1f-b6453eb207f9, 6, Finished, Available)

'1900-01-01'

In [5]:
dataframe_bze=spark.read.table("TMDW_FinalProj.bronzesales")

dataframe_final=dataframe_bze.selectExpr("Cust_ID","Cust_Name","Segment","City","State", \
                   "Country","Region")\
                   .where(col("Modified_TS")>Maximum_Date)\
                   .drop_duplicates()

StatementMeta(, 30bc0446-b872-4379-bd1f-b6453eb207f9, 7, Finished, Available)

In [6]:
dataframe_final.createOrReplaceTempView("View_Cust")

StatementMeta(, 30bc0446-b872-4379-bd1f-b6453eb207f9, 8, Finished, Available)

In [7]:
%%sql
Merge into TMDW_FinalProj.gld_Cust as gc
using View_Cust as vc
on gc.Cust_ID=vc.Cust_ID
when matched then 
update set
gc.Cust_Name	=	vc.Cust_Name	,
gc.Segment	=	vc.Segment	,
gc.City	=	vc.City	,
gc.State	=	vc.State	,
gc.Country	=	vc.Country	,
gc.Region	=	vc.Region	,
gc.Modified_TS	=	current_timestamp()	

when not matched then insert
(
gc.Cust_ID	,
gc.Cust_Name	,
gc.Segment	,
gc.City	,
gc.State	,
gc.Country	,
gc.Region	,
gc.Created_TS	,
gc.Modified_TS	
)
values
(
vc.Cust_ID	,
vc.Cust_Name	,
vc.Segment	,
vc.City	,
vc.State	,
vc.Country	,
vc.Region	,
current_timestamp(),
current_timestamp
)

StatementMeta(, 30bc0446-b872-4379-bd1f-b6453eb207f9, 9, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

In [8]:
%%sql
select * from gld_cust limit 15

StatementMeta(, 30bc0446-b872-4379-bd1f-b6453eb207f9, 10, Finished, Available)

<Spark SQL result set with 15 rows and 9 fields>